<a href="https://colab.research.google.com/github/nulven/PrivacyML/blob/master/Audio_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
%cd '/content/drive/MyDrive/csail/mnist'

/content/drive/MyDrive/csail/mnist


Preprocess data

In [41]:
%matplotlib inline
import time
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import os
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models

In [42]:
np.random.seed(0)

In [43]:
import librosa
audio_folder = '/content/drive/MyDrive/csail/mnist/audio_mnist'

audio = listdir(audio_folder)
# neg_audio = listdir(neg_audio_folder)

len(audio)

3000

In [79]:
import IPython.display as ipd
ipd.Audio(join(audio_folder,audio[7]))

In [82]:
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display

In [83]:
#Audio Plots
from tqdm import tqdm 
import time 
'''
for aud in tqdm(audio):
  x , sr = librosa.load(audio_folder+'/'+aud)
  mfccs = librosa.feature.mfcc(x, sr=sr)
  fig = plt.Figure()
  canvas = FigureCanvas(fig)
  ax = fig.add_subplot(111)
  librosa.display.specshow(mfccs, sr=sr)
  plt.margins(0,0)
  plt.axis('off')
  plt.savefig('/content/drive/MyDrive/csail/mnist/audio_plots/' + aud + '.png',bbox_inches='tight',pad_inches = 0)
  # time.sleep(0.01)
'''

"\nfor aud in tqdm(audio):\n  x , sr = librosa.load(audio_folder+'/'+aud)\n  mfccs = librosa.feature.mfcc(x, sr=sr)\n  fig = plt.Figure()\n  canvas = FigureCanvas(fig)\n  ax = fig.add_subplot(111)\n  librosa.display.specshow(mfccs, sr=sr)\n  plt.margins(0,0)\n  plt.axis('off')\n  plt.savefig('/content/drive/MyDrive/csail/mnist/audio_plots/' + aud + '.png',bbox_inches='tight',pad_inches = 0)\n  # time.sleep(0.01)\n"

In [84]:
#Dataset Path
images = '/content/drive/MyDrive/csail/mnist/audio_plots'

In [85]:
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
ds_trans = transforms.Compose([transforms.Resize(224),
                               transforms.ToTensor(),
                               normalize])

In [114]:
files = os.listdir('/content/drive/MyDrive/csail/mnist/audio_mnist')
digits = [int(file[0]) for file in files]
names = [file.split('_')[1] for file in files]
d = {'fname': files, 'label': digits, 'name': names}
labels = pd.DataFrame(data=d)

In [87]:
#Viewing Image
from IPython.display import Image
# Image(images+'pos-0422-096-cough-m-31-8.mp3.png')

In [88]:
from torch.utils.data import Dataset
import pandas as pd
import os
from PIL import Image
import torch

class HeartbeatDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = annotation_file
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        img = Image.open(os.path.join(self.root_dir, img_id+'.png')).convert("RGB")
        y_label = torch.tensor(float(self.annotations.iloc[index, 1])).type(torch.LongTensor)

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)


normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
ds_trans = transforms.Compose([transforms.Resize(224),
                               transforms.CenterCrop(224),
                               transforms.ToTensor(),
                               normalize])

In [89]:
dataset = HeartbeatDataset("/content/drive/MyDrive/csail/mnist/audio_plots",labels,transform=ds_trans)

train_set, validation_set = torch.utils.data.random_split(dataset,[2000, 1000])

train_dl = DataLoader(dataset=train_set, shuffle=True, batch_size=4,num_workers=4)
valid_dl = DataLoader(dataset=validation_set, shuffle=True, batch_size=4,num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [92]:
use_gpu = torch.cuda.is_available()
resnet = models.resnet50(pretrained=True)
inputs, labels = next(iter(train_dl))
if use_gpu:
    resnet = resnet.cuda()
    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())   
else:
    inputs, labels = Variable(inputs), Variable(labels)
outputs = resnet(inputs)
outputs.size()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([4, 1000])

In [93]:
def train_model(dataloders, model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    use_gpu = torch.cuda.is_available()
    best_model_wts = model.state_dict()
    best_acc = 0.0
    dataset_sizes = {'train': len(dataloders['train'].dataset), 
                     'valid': len(dataloders['valid'].dataset)}

    for epoch in range(num_epochs):
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloders[phase]:
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                optimizer.zero_grad()

                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)

                # print("F1 Score = {}, Precision = {}, Recall = {}".format(f1(labels,preds),precision(labels,preds),recall(labels,preds)))

                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                train_epoch_loss = running_loss / dataset_sizes[phase]
                train_epoch_acc = running_corrects / dataset_sizes[phase]
            else:
                valid_epoch_loss = running_loss / dataset_sizes[phase]
                valid_epoch_acc = running_corrects / dataset_sizes[phase]
                
            if phase == 'valid' and valid_epoch_acc > best_acc:
                best_acc = valid_epoch_acc
                best_model_wts = model.state_dict()

        print('Epoch [{}/{}] train loss: {:.4f} acc: {:.4f} ' 
              'valid loss: {:.4f} acc: {:.4f}'.format(
                epoch, num_epochs - 1,
                train_epoch_loss, train_epoch_acc, 
                valid_epoch_loss, valid_epoch_acc))
            
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [94]:
resnet = models.resnet50(pretrained=True)
# freeze all model parameters
for param in resnet.parameters():
    param.requires_grad = False

# new final layer with 2 classes
num_ftrs = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs, 10)
CUDA_LAUNCH_BLOCKING=1
if use_gpu:
    resnet = resnet.cuda()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet.fc.parameters(), lr=0.05, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

dloaders = {'train':train_dl, 'valid':valid_dl}

In [56]:
#start_time = time.time()
#model = train_model(dloaders, resnet, criterion, optimizer, exp_lr_scheduler, num_epochs=20)
#print('Training time: {:10f} minutes'.format((time.time()-start_time)/60))

In [57]:
!pip install pytorch-lightning

In [95]:
from pytorch_lightning.metrics.classification import Precision, Recall, F1

precision = Precision(num_classes = 10)
recall = Recall()
f1 = F1(num_classes = 10)

In [96]:
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display

In [97]:
import numpy as np 
import matplotlib.pyplot as plt
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR, CosineAnnealingWarmRestarts
import torchvision
import torchvision.transforms as tt
import torchvision.models as models
from torchvision.datasets import MNIST, CIFAR10, ImageFolder
from torchvision.utils import make_grid
from torch.utils.data import random_split, DataLoader, Subset
from torch.nn.utils import vector_to_parameters, parameters_to_vector

from sklearn.metrics import accuracy_score

import os
import time
import pickle

torch.manual_seed(0)
np.random.seed(0)

In [98]:
batch_size=100

In [101]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
device = 'cpu'
device

'cpu'

In [63]:
class MnistNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1, 10, 5),
                                   nn.MaxPool2d(2),
                                   nn.ReLU())
        self.conv2 = nn.Sequential(nn.Conv2d(10, 20, kernel_size=5),
                                   nn.Dropout2d(),
                                   nn.MaxPool2d(2),
                                   nn.ReLU())
        self.fc1 = nn.Sequential(nn.Flatten(),
                                 nn.Linear(320, 50),
                                 nn.Dropout(),
                                 nn.ReLU())
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [102]:
def train(model, train_dl, optimizer, criterion):
    model.to(device)
    model.train()
    batch_loss, batch_acc = [], []
    for images, labels in train_dl:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        batch_loss.append(loss.detach().cpu())
        pred = torch.argmax(logits, dim=1)
        batch_acc.append(accuracy_score(labels.cpu(), pred.cpu()))
    model.cpu()
    return sum(batch_loss)/len(batch_loss), sum(batch_acc)/len(batch_acc)

In [103]:
def test(model, test_dl, criterion):      
    with torch.no_grad():
        model.to(device)
        model.eval()
        batch_loss, batch_acc = [], []
        for images, labels in test_dl:
            images, labels = images.to(device), labels.to(device)
            logits = model(images)
            loss = criterion(logits, labels)
            batch_loss.append(loss.cpu())
            pred = torch.argmax(logits, dim=1)
            batch_acc.append(accuracy_score(labels.cpu(), pred.cpu()))
        model.cpu()
        return sum(batch_loss)/len(batch_loss), sum(batch_acc)/len(batch_acc)

In [104]:
def fit(epochs, model, optimizer,criterion, train_dl, test_dl):
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for epoch in range(1,epochs+1):
        trainl, traina = train(model, train_dl, optimizer, criterion)
        testl , testa = test(model, test_dl, criterion)
        train_loss.append(trainl)
        train_acc.append(traina)
        test_loss.append(testl)
        test_acc.append(testa)
        print(f'Epoch {epoch} - train_loss : {trainl :.4f}, train_acc : {traina:.4f}, test_loss : {testl:.4f}, test_acc : {testa:0.4f}')

    history = {'train_loss' : train_loss,
               'train_acc' : train_acc,
               'test_loss' : test_loss,
               'test_acc' : test_acc}
    return history

In [105]:
def plot(epochs, title, history):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))
    t = f.suptitle(title, fontsize=24)
    f.patch.set_facecolor('white')
    f.subplots_adjust(top=0.85, wspace=0.3)
    epoch_list = list(range(1,epochs+1))

    ax1.plot(epoch_list, history['train_acc'], label='Train Accuracy')
    ax1.plot(epoch_list, history['test_acc'], label='Test Accuracy')
    ax1.set_ylabel('Accuracy Value')
    ax1.set_xlabel('Epoch')
    ax1.set_title('Accuracy', fontsize = 18)
    ax1.grid(True)
    l1 = ax1.legend(loc="best")

    ax2.plot(epoch_list, history['train_loss'], label='Train Loss')
    ax2.plot(epoch_list, history['test_loss'], label='Test Loss')
    ax2.set_ylabel('Loss Value')
    ax2.set_xlabel('Epoch')
    ax2.set_title('Loss', fontsize = 18)
    ax2.grid(True)
    l2 = ax2.legend(loc="best")
    return f

In [106]:
def save(history, model, figure, name):
    with open(name + '.pickle', 'wb') as handle:
        pickle.dump(history, handle)
    torch.save(model, name + '.pth')
    figure.savefig(name + '.png')

In [107]:
def make_iid_dls(train_ds, n):
    size = len(train_ds)//n
    last_size = size + len(train_ds)%n
    client_ds = random_split(train_ds, [size]*(n-1) + [last_size])
    client_dls = [DataLoader(ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)for ds in client_ds]
    return client_dls

In [108]:
def make_noniid_dls(train_ds, n):
    num_shards, num_imgs = 2*n, len(train_ds)//(2*n)
    idx_shard = [i for i in range(num_shards)]
    dict_clients = {i: np.array([]) for i in range(n)}
    idxs = np.arange(num_shards*num_imgs)
    labels = train_ds.targets

    # sort labels
    idxs_labels = np.vstack((idxs, labels[:len(idxs)]))
    idxs_labels = idxs_labels[:,idxs_labels[1,:].argsort()]
    idxs = idxs_labels[0,:]

    # divide and assign
    for i in range(n):
        rand_set = set(np.random.choice(idx_shard, 2, replace=False))
        idx_shard = list(set(idx_shard) - rand_set)
        for rand in rand_set:
            dict_clients[i] = np.concatenate((dict_clients[i], idxs[rand*num_imgs:(rand+1)*num_imgs]), axis=0)

    client_dls = []
    for i in range(n):
        ds = Subset(train_ds, torch.LongTensor(dict_clients[i]))
        client_dls.append(DataLoader(ds, batch_size, shuffle = True, num_workers = 4, pin_memory = True))
    return client_dls

In [109]:
def make_iid_dls_with_speakers(train_ds, n):
    size = len(train_ds)//n
    last_size = size + len(train_ds)%n
    train_ds = train_ds.reset_index(drop=True)
    idxs = train_ds.index
    client_idxs = random_split(list(idxs), [size]*(n-1) + [last_size])
    print(client_idxs[0])
    client_ds = [train_ds.iloc[list(idxs)] for idxs in client_idxs]
    client_ds = [HeartbeatDataset("/content/drive/MyDrive/csail/mnist/audio_plots",ds,transform=ds_trans) for ds in client_ds]
    client_dls = [DataLoader(ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)for ds in client_ds]
    return client_dls

In [110]:
def make_noniid_dls_with_speakers(train_ds, n=6):
    client_dls = []
    for name in set(train_ds.name):
      subset = train_ds[train_ds.name == name]
      subset = subset.reset_index(drop=True)
      ds = HeartbeatDataset("/content/drive/MyDrive/csail/mnist/audio_plots",subset,transform=ds_trans)
      client_dls.append(DataLoader(ds, batch_size, shuffle = True, num_workers = 4, pin_memory = True))
   
    return client_dls

In [115]:
dataset = HeartbeatDataset("/content/drive/MyDrive/csail/mnist/audio_plots",labels,transform=ds_trans)

train_idxs = []
validation_idxs = []
for name in set(labels.name):
  for digit in range(10):
    idxs = labels[labels.name == name]
    idxs = idxs[idxs.label == digit].index
    train_idxs1, validation_idxs1 = torch.utils.data.random_split(list(idxs), [30, 20])
    train_idxs.extend(list(train_idxs1))
    validation_idxs.extend(list(validation_idxs1))
train_set, validation_set = torch.utils.data.Subset(dataset, train_idxs), torch.utils.data.Subset(dataset, validation_idxs)
train_ds, validation_ds = labels.iloc[train_idxs], labels.iloc[validation_idxs]

train_dl = DataLoader(dataset=train_set, shuffle=True, batch_size=4,num_workers=4)
validation_dl = DataLoader(dataset=validation_set, shuffle=True, batch_size=4,num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [117]:
torch.manual_seed(0)
np.random.seed(0)
n = 6
iid_dls = make_iid_dls_with_speakers(train_ds, n)
noniid_dls = make_noniid_dls_with_speakers(train_ds, n)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [118]:
def make_clients(n):
    models = [resnet for _ in range(n)]
    optimizers = [SGD(model.parameters(), lr=1e-2, momentum=0.9) for model in models]
    return models, optimizers

In [119]:
def train_clients(client_models, client_optimizers, server_model, criterion, client_dls):
    client_loss, client_acc = [], []
    for model, optimizer, train_dl in zip(client_models, client_optimizers, client_dls):
        model.load_state_dict(server_model.state_dict())
        closs, cacc = train(model, train_dl, optimizer, criterion)
        client_loss.append(closs)
        client_acc.append(cacc)
    return sum(client_loss)/len(client_loss), sum(client_acc)/len(client_acc)

In [120]:
def compute_gradient(client_models, server_model, r, compression,sparsity, warmup, epoch):
    n = len(client_models)
    server_dict = server_model.state_dict()
    grad_dict = {k : torch.zeros_like(v) for k, v in server_dict.items()}
    if compression is None:
        for model in client_models:
            client_dict =  model.state_dict()
            for name in client_dict:
                grad_dict[name] += client_dict[name]
        grad_dict = {k : v/n-server_dict[k] for k,v in grad_dict.items()}
    else:
        if epoch <= warmup:
            scale = (sparsity/(1-1/n))**(1/warmup) if compression == 'rTopk' else (sparsity/0.00001)**(1/warmup)
            sparsity /= scale**(warmup-epoch+1)
        grad_vector = nn.utils.parameters_to_vector(grad_dict.values())
        count_vector = copy.deepcopy(grad_vector)
        for i, model in enumerate(client_models):
            client_dict =  model.state_dict()
            residual_dict = r[i]
            g = {k : client_dict[k] - server_dict[k] + residual_dict[k] for k in client_dict}
            g_vector = nn.utils.parameters_to_vector(g.values())
            if compression == "rTopk":
                _, rtop_idx = torch.topk(g_vector.abs(), int((1-sparsity)* len(g_vector))*n, sorted = False)
                topk_idx = np.random.choice(rtop_idx, len(rtop_idx)//n, replace = False)
            else:
                _, topk_idx = torch.topk(g_vector.abs(), int((1-sparsity)* len(g_vector)), sorted = False)
            residual_vector = copy.deepcopy(g_vector)
            residual_vector[topk_idx] = 0
            nn.utils.vector_to_parameters(residual_vector, residual_dict.values())
            grad_vector[topk_idx] += g_vector[topk_idx]
            count_vector[topk_idx] += 1
        count_vector[count_vector == 0] = 1
        grad_vector /= count_vector
        nn.utils.vector_to_parameters(grad_vector, grad_dict.values())
    return grad_dict

In [121]:
def train_server(model, opt, lr, g, m, v, beta1, beta2, epsilon, epoch, demon, total_epochs):
    s_dict = model.state_dict()
    if opt == 'sgd':
        new_s_dict = {k : s_dict[k] + lr * g[k] for k in s_dict}
    else:
        if demon:
            p = (1-epoch/total_epochs) 
            beta1 *= p/(1-beta1 + beta1*p)
            m = {k : beta1*m[k] + g[k] for k in g}
        else:
            m = {k : beta1*m[k] + (1-beta1)*g[k] for k in g}
        if opt == 'adagrad':
            v =  {k : v[k] + g[k]**2 for k in v}
        if opt == 'yogi':
            v = {k : v[k] - (1-beta2)*torch.sign(v[k] - g[k]**2)*(g[k]**2) for k in v}
        if opt == 'adam':
            v = {k : beta2*v[k] + (1-beta2)*(g[k]**2) for k in v}
        lr *= ((1-beta2**epoch)**0.5)/(1-beta1**epoch)
        new_s_dict = {k : s_dict[k] + lr * m[k]/(v[k]**0.5 + epsilon) for k in s_dict}
    model.load_state_dict(new_s_dict)
    return m, v

In [122]:
def fit_federated(epochs, client_models, client_optimizers, client_dls, 
                  server_model, server_opt, server_lr, test_dl, criterion,
                  beta1 = 0, beta2 = 0, epsilon = 1e-8, demon = False, compression = None, sparsity = 0, warmup = 5):
    s_dict = server_model.state_dict()
    m = {k : torch.zeros_like(v) for k, v in s_dict.items()}
    v = copy.deepcopy(m)
    r = [copy.deepcopy(m) for _ in range(len(client_models))]
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for epoch in range(1,epochs+1):
        trainl, traina = train_clients(client_models, client_optimizers, server_model, criterion, client_dls)
        g = compute_gradient(client_models, server_model, r, compression, sparsity, warmup, epoch)
        m, v = train_server(server_model, server_opt, server_lr, g, m, v, beta1, beta2, epsilon, epoch, demon, epochs)
        testl , testa = test(server_model, test_dl, criterion)
        train_loss.append(trainl)
        train_acc.append(traina)
        test_loss.append(testl)
        test_acc.append(testa)
        print(f'Epoch {epoch} - train_loss : {trainl :.4f}, train_acc : {traina:.4f}, test_loss : {testl:.4f}, test_acc : {testa:0.4f}')

    history = {'train_loss' : train_loss,
               'train_acc' : train_acc,
               'test_loss' : test_loss,
               'test_acc' : test_acc}
    return history

In [123]:
epochs = 250
criterion = nn.CrossEntropyLoss()

In [125]:
client_models, client_optimizers = make_clients(n)
server_model = resnet
server_opt = 'sgd'
server_lr = 1

In [ ]:
iid_sgd_history = fit_federated(epochs, client_models, client_optimizers, iid_dls, 
                                server_model, server_opt, server_lr, validation_dl, criterion)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 1 - train_loss : 2.2689, train_acc : 0.1589, test_loss : 2.2836, test_acc : 0.1158


In [ ]:
f = plot(epochs, "IID SGD Performace", iid_sgd_history)

In [ ]:
save(iid_sgd_history, server_model, f, "IID_SGD")

In [ ]:
client_models, client_optimizers = make_clients(n)
server_model = MnistNet()

In [ ]:
noniid_sgd_history = fit_federated(epochs, client_models, client_optimizers, noniid_dls, 
                                server_model, server_opt, server_lr, test_dl, criterion)

In [ ]:
f = plot(epochs, "NonIID-SGD Performance", noniid_sgd_history)

In [ ]:
save(noniid_sgd_history, server_model, f, "NonIID_SGD")

In [ ]:
client_models, client_optimizers = make_clients(n)
server_model = MnistNet()
server_opt = 'adam'
server_lr = 0.003
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-8
demon = True

In [ ]:
noniid_demon_adam_history = fit_federated(epochs, client_models, client_optimizers, noniid_dls, 
                  server_model, server_opt, server_lr, test_dl, criterion,
                  beta1, beta2, epsilon, demon)

In [ ]:
f = plot(epochs, "NonIID Demon Adam Performace", noniid_demon_adam_history)

In [ ]:
save(noniid_demon_adam_history, server_model, f, "NonIID_Demon_Adam")

In [ ]:
client_models, client_optimizers = make_clients(n)
server_model = MnistNet()
compression = 'rTopk'
server_lr = 0.001
sparsity = 0.99
warmup = 5

In [ ]:
noniid_demon_adam_rtopk_history = fit_federated(epochs, client_models, client_optimizers, noniid_dls, 
                  server_model, server_opt, server_lr, test_dl, criterion,
                  beta1, beta2 , epsilon , demon , compression , sparsity , warmup)

In [ ]:
f = plot(epochs, "NonIID Demon Adam with 99% rTopk Compression Performace", noniid_demon_adam_rtopk_history)

In [ ]:
save(noniid_demon_adam_rtopk_history, server_model, f, "NonIID_Demon_Adam_rTopk")

In [ ]:
client_models, client_optimizers = make_clients(n)
server_model = MnistNet()
server_opt = 'sgd'
server_lr = 0.1
compression = 'rTopk'
sparsity = 0.99
warmup = 5

In [ ]:
noniid_sgd_rtopk_history = fit_federated(epochs, client_models, client_optimizers, noniid_dls, 
                  server_model, server_opt, server_lr, test_dl, criterion,
                  compression = compression , sparsity = sparsity , warmup = warmup)

In [ ]:
f = plot(epochs, "NonIID SGD with 99% rTopk Compression Performace", noniid_sgd_rtopk_history)

In [ ]:
save(noniid_sgd_rtopk_history, server_model, f, "NonIID_SGD_rTopk")